<CENTER><img src="http://opendata.atlas.cern/DataAndTools/pictures/ATLASOD.gif" style="width:50%"></CENTER>

<CENTER><h1>Searching for the Higgs boson in the H&#8594;&gamma;&gamma; channel</h1></CENTER>
## Python notebook example

**Introduction**
Let's take a current ATLAS Open Data sample and create a histogram:

In [1]:
# We added all data sets
# This Code Must be modified for a faster ploting!
# Thaks to Mohammad and Arturo (& Leo + Clement!)
import ROOT
from ROOT import TMath, TChain
import time

Welcome to ROOTaaS 6.06/08


In [2]:
start = time.time()

Because we would like to use more than one ROOT input file, the best option is to use a TChain object. This allows to "chain" several samples into a single structure that we can later loop over

In [3]:
#f = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/2019/GamGam/Data/dataB_yy.root")
    #("Data/dataA_yy.root")
    #("Data/dataB_yy.root")
    #("Data/dataC_yy.root")
    #("Data/dataD_yy.root")
chain = TChain("mini")    
chain.Add("http://opendata.atlas.cern/release/samples/2019/GamGam/Data/dataA_yy.root")
chain.Add("http://opendata.atlas.cern/release/samples/2019/GamGam/Data/dataB_yy.root")
chain.Add("http://opendata.atlas.cern/release/samples/2019/GamGam/Data/dataC_yy.root")
chain.Add("http://opendata.atlas.cern/release/samples/2019/GamGam/Data/dataD_yy.root")

chain.SetBranchStatus("*",0)
chain.SetBranchStatus("photon_pt",1)
chain.SetBranchStatus("photon_eta",1)
chain.SetBranchStatus("photon_ptcone30",1)
chain.SetBranchStatus("photon_etcone20",1)
chain.SetBranchStatus("photon_phi",1)
chain.SetBranchStatus("photon_E",1)
chain.SetBranchStatus("trigP",1)
chain.SetBranchStatus("photon_n",1)
chain.SetBranchStatus("photon_isTightID",1)



In [4]:
#tree = f.Get("mini")

Now we're going to extract the photons variables

In [5]:
#Invariant mass histograms definition
hist = ROOT.TH1F("h_M_Hyy","Invariant Mass of di-photon system ; Invariant Mass m_{yy} [GeV] ; events",50,80,280)

Here we're filling the variables defined above with the content of those inside the input ntuples

We're creating a histogram for this example. The plan in to fill them with events.

We are selecting below a simple look for them.

The Higgs boson analysis implemented here considers Higgs boson decays into a proton-proton pair. The event selection criteria are:

In [6]:
Photon_1 = ROOT.TLorentzVector()
Photon_2 = ROOT.TLorentzVector()
n = 0
for event in chain:
    n += 1
    ## printing the evolution in number of events
    if(n%1000==0):
        print(n)
    ## checking the trigger    
    if(chain.trigP):
        goodphoton_index = [0]*5
        goodphoton_n = 0
        photon_index = 0
        ##            
        j=0
        ## looping the photons per event
        ## 25000 changed to 40000
        for j in range(chain.photon_n):
            ##
            if(chain.photon_isTightID[j]):
                ##
                if(chain.photon_pt[j] > 25000 and (TMath.Abs(chain.photon_eta[j]) < 2.37)\
                   and (TMath.Abs(chain.photon_eta[j]) < 1.37 or TMath.Abs(chain.photon_eta[j]) > 1.52)):
                    ##
                    goodphoton_n += 1  #count
                    goodphoton_index[photon_index]=j
                    photon_index += 1
                ## end Pt and eta pre-selection
            ## end on request of quality of the photon
        ## end looping photons in the current event
            
        ## Using the two selected photons
        if(goodphoton_n==2):
            ##
            goodphoton1_index = goodphoton_index[0]
            goodphoton2_index = goodphoton_index[1]
            ## Getting couple of photons with good isolation 
            if((chain.photon_ptcone30[goodphoton1_index]/chain.photon_pt[goodphoton1_index] < 0.065)\
               and (chain.photon_etcone20[goodphoton1_index] / chain.photon_pt[goodphoton1_index] < 0.065)):
                ##
                if((chain.photon_ptcone30[goodphoton2_index]/chain.photon_pt[goodphoton2_index] < 0.065)\
                   and (chain.photon_etcone20[goodphoton2_index] / chain.photon_pt[goodphoton2_index] < 0.065)):
                    ##
                    Photon_1.SetPtEtaPhiE(chain.photon_pt[goodphoton1_index]/1000., chain.photon_eta[goodphoton1_index],\
                                          chain.photon_phi[goodphoton1_index],chain.photon_E[goodphoton1_index]/1000.)
                    Photon_2.SetPtEtaPhiE(chain.photon_pt[goodphoton2_index]/1000., chain.photon_eta[goodphoton2_index],\
                                          chain.photon_phi[goodphoton2_index],chain.photon_E[goodphoton2_index]/1000.)
                    ## Adding the two TLorentz vectors
                    Photon_12 = Photon_1 + Photon_2
                    ## Filling with the mass of the gamma-gamma system
                    hist.Fill(Photon_12.M())
                ## end isolation photon #2
            ## end isolation photon #1
        ## end 2-good photons
    ## end of trigger request
## End loop in the events

1000


RuntimeError: TTree I/O error

Error in <TWebFile::GetFromWeb10>: expected -2975568 bytes, got 0
Error in <TBranchElement::GetBasket>: File: http://opendata.atlas.cern/release/samples/2019/GamGam/Data/dataA_yy.root at byte:222, branch:photon_ptcone30, entry:1226, badread=1, nerrors=1, basketnumber=1


#### Final plot

In [ ]:
canvas = ROOT.TCanvas("Canvas","cz",1000,1000)
hist.Draw()
canvas.Draw()

#### Log Scale

In [ ]:
hist.Draw("E")
hist.SetMinimum(10)
canvas.SetLogy()
canvas.Draw()

In [ ]:
end = time.time()
duration = end-start
print("Finished in {} min {} s").format(int(duration//60),int(duration%60))